In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

LOADING DATASET - UCI - BREAST CANCER (WISCONSIN)

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"

names = ['id', 'clump_thickness','uniform_cell_size','uniform_cell_shape','marginal_adhesion',
         'single_epi_size','bare_nuclei','bland_cromatin','normal_nucleoli',
         'mitosis','class']

df = pd.read_csv(url,names=names)

In [ ]:
df.head()

In [ ]:
df.shape

**Data Preprocessing**

In [ ]:
df.drop(['id'],axis = 1,inplace = True)

In [ ]:
df.head()

**MISSING VALUES**

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df['class'].value_counts()

In [ ]:
df['bare_nuclei'].value_counts()

In [ ]:
df[df['bare_nuclei']=="?"]

In [ ]:
df[df['bare_nuclei']=="?"].sum()

In [ ]:
df.replace("?",np.nan,inplace=True)

In [ ]:
df['bare_nuclei'][23]

In [ ]:
df.isna().sum()

In [ ]:
df.fillna(method='ffill',inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df['bare_nuclei'] = df['bare_nuclei'].astype('int64')

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
sns.distplot(df['class'],kde = True)

In [ ]:
sns.set_style("darkgrid")
df.hist(figsize=(30,30))
plt.show()

In [ ]:
scatter_matrix(df,figsize=(20,20))
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(df,orient='h')

In [ ]:
sns.pairplot(df,diag_kind='kde')

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(15,15))
cor = df.corr()
sns.heatmap(cor,annot=True)
plt.title("Correlation b/w different values")
plt.show()

In [ ]:
cor_target = abs(cor["class"])

relevant_features = cor_target[cor_target>0]
relevant_features

#Train Test Split model

In [ ]:
Y = df['class'].values
X = df.drop('class',axis = 1).values

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.30,random_state=21)

Model Selection

In [ ]:
scoring = 'accuracy'

models = []
models.append(("CART", DecisionTreeClassifier()))
models.append(("SVM", SVC()))
models.append(("NB", GaussianNB()))
models.append(("KNN", KNeighborsClassifier()))

#evaluate
results = []
names = []

for name,model in models:
  kfold = KFold(n_splits = 10)
  cvresult = cross_val_score(model,X_train,y_train,cv=kfold,scoring = scoring)
  results.append(cvresult)
  names.append(name)
  msg = "For %s Model:Mean accuracy is %f(Std accuracy is %f)" %(name,cvresult.mean(),cvresult.std())
  print(msg)

#Validation test

In [ ]:
for name,model in models:
  model.fit(X_train, y_train)
  predictions = model.predict(X_test)
  print("Model name:",name)
  print("Accuracy Score:", accuracy_score(y_test,predictions))
  print("Classification Report", classification_report(y_test, predictions))

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.suptitle("Performance Comparison of the Models")
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

#Support Vector Machines - Best performance


In [ ]:
clf = SVC()

clf.fit(X_train, y_train)

accuracy = clf.score(X_test, y_test)

print("Accuracy of SVM is: ", accuracy)

In [ ]:
predict = clf.predict(X_test)
predict

In [ ]:
df.info()

In [ ]:
df.tail()

In [ ]:
example_measures = [[4,2,1,1,1,2,3,2,1]]

prediction = clf.predict(example_measures)
print("The Class is (2 for Benign and 4 for Malign): ", prediction)

In [ ]:
example_m2 = [[5,10,10,3,7,3,8,10,2]]
prediction = clf.predict(example_m2)
print("The Class is: ", prediction)	

#Pickle file getting downloaded

In [ ]:
import pickle

pickle.dump(clf,open('modelsvm.pkl','wb'))

model = pickle.load(open('modelsvm.pkl','rb'))
print(model.predict([[4,2,1,1,1,2,3,2,1]]))